In [1]:
import shutil

shutil.copytree("/kaggle/input/fine-tune-bloom-560m-for-sumarization/transformers/default/1/", "model")

'model'

In [2]:
pip install -U -q transformers datasets peft evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 5.5 MB/s eta 0:00:00
E

In [3]:
from datasets import load_dataset

dataset = load_dataset("OpenHust/vietnamese-summarization", split="train")

README.md:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Kmeans_1024_new.csv:   0%|          | 0.00/32.3M [00:00<?, ?B/s]

Kmeans_512_new.csv:   0%|          | 0.00/25.0M [00:00<?, ?B/s]

Kmeans_512_token_new.csv:   0%|          | 0.00/29.1M [00:00<?, ?B/s]

bio_medicine.csv:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

herding_512_bio_medicine.csv:   0%|          | 0.00/25.1M [00:00<?, ?B/s]

herding_bio_medicine.csv:   0%|          | 0.00/32.3M [00:00<?, ?B/s]

herding_prompt_512_bio_medicine.csv:   0%|          | 0.00/29.1M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
from transformers import AutoTokenizer

model_id = "bigscience/bloom-560m"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token  # required for BLOOM
dataset = dataset.rename_columns({"Document": "text", "Summary": "summary"})
dataset = dataset.remove_columns(["Unnamed: 0", "Dataset"])
max_length = 768
def is_short_enough(example):
    prompt = f"Tóm tắt văn bản sau:\n{example['text']}\nTóm tắt:"
    full_text = prompt + example['summary']
    tokenized = tokenizer(full_text, truncation=False)
    return len(tokenized["input_ids"]) <= 768

dataset = dataset.filter(is_short_enough)
print(len(dataset))
# Optional: use a subset for speed (adjust number as needed)
subDataset = dataset.shuffle(seed=666).select(range(15000))

# Split into train/validation
train_test_split = subDataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Filter:   0%|          | 0/74564 [00:00<?, ? examples/s]

62070


In [5]:
import torch
import evaluate
from typing import Union, List
from transformers import PreTrainedModel, PreTrainedTokenizer
from tqdm import tqdm
# Load metrics once
_rouge = evaluate.load("rouge")
_meteor = evaluate.load("meteor")

def evaluate_summarization_model(
    model: PreTrainedModel,
    tokenizer: PreTrainedTokenizer,
    dataset,
    prompt_template: str = "Tóm tắt văn bản sau:\n{text}\nTóm tắt:",
    device: str = "cuda",
    max_input_length: int = 768,
    max_new_tokens: int = 128,
    num_samples: Union[int, None] = None,
) -> dict:
    """
    Compute ROUGE and METEOR scores of `model` on `dataset`.

    Args:
      model: a Hugging Face causal LM or PeftModelForCausalLM
      tokenizer: its tokenizer
      dataset: a Hugging Face Dataset with 'text' and 'summary' fields
      prompt_template: how to wrap `text` into the generation prompt
      device: "cuda" or "cpu"
      max_input_length: tokens to truncate+pad the prompt
      max_new_tokens: how many tokens to generate
      num_samples: if set, only evaluate the first N examples

    Returns:
      A dict with keys "rouge1", "rouge2", "rougeL" and "meteor"
    """
    model.to(device)
    model.eval()

    preds: List[str] = []
    refs:  List[str] = []

    for i, ex in tqdm(enumerate(dataset)):
        if num_samples is not None and i >= num_samples:
            break

        # prepare input
        text = ex["text"]
        ref  = ex["summary"]
        prompt = prompt_template.format(text=text)
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=max_input_length
        ).to(device)

        # generate
        with torch.no_grad():
            out = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False
            )
        decoded = tokenizer.decode(out[0], skip_special_tokens=True)
        # strip off the prompt itself
        pred = decoded.split("Tóm tắt:")[-1].strip()

        preds.append(pred)
        refs.append(ref)

    # compute metrics
    rouge_res  = _rouge.compute(predictions=preds, references=refs, use_stemmer=True)
    meteor_res = _meteor.compute(predictions=preds, references=refs)

    return {
        "rouge1": round(rouge_res["rouge1"] * 100, 2),
        "rouge2": round(rouge_res["rouge2"] * 100, 2),
        "rougeL": round(rouge_res["rougeL"] * 100, 2),
        "meteor": round(meteor_res["meteor"] * 100, 2),
    }


def main(data):
    from transformers import AutoModelForCausalLM, AutoTokenizer
    from peft import PeftModel
    device_map={'':torch.cuda.current_device()}
    # original model
    tok = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
    base = AutoModelForCausalLM.from_pretrained("bigscience/bloom-560m", device_map=device_map)
    orig_scores = evaluate_summarization_model(base, tok, data)

    # LoRA-fine-tuned model
    finetuned = PeftModel.from_pretrained(base, "./model").to("cuda")
    lora_scores = evaluate_summarization_model(finetuned, tok, data)

    print("Original model:", orig_scores)
    print("LoRA-tuned model:", lora_scores)


2025-05-08 14:24:36.871473: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746714277.056030      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746714277.110582      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [6]:
main(eval_dataset)

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

1500it [1:14:49,  2.99s/it]
1500it [1:28:14,  3.53s/it]


Original model: {'rouge1': 38.01, 'rouge2': 14.42, 'rougeL': 24.97, 'meteor': 21.58}
LoRA-tuned model: {'rouge1': 33.73, 'rouge2': 14.03, 'rougeL': 24.77, 'meteor': 24.63}
